In [ ]:
!pip install pulp # Instalacja biblioteki Pulp do rozwiązywania problemów optymalizacyjnych
from pulp import * # Importowanie wszystkich funkcji z biblioteki Pulp

# Definiowanie wartości napięcia dla każdej gałęzi obwodu (w Voltach)
voltage = [6, 10, 4, 7, 3]
voltage_squared = [v**2 for v in voltage] # Obliczenie kwadratu napięcia dla każdej gałęzi (używane później do obliczania mocy)

# Definiowanie maksymalnego prądu dozwolonego dla każdego rezystora (w Amperach)
max_currents = [5, 3, 3, 3, 5]
# Definiowanie minimalnego prądu dozwolonego dla każdego rezystora (w Amperach)
min_currents = [3, 1, 1, 1, 3]

# Definiowanie konduktancji dla każdej gałęzi jako stosunek minimalnego prądu do napięcia
conductance = [min_currents[i]/voltage[i] for i in range(5)]

# Tworzenie problemu optymalizacyjnego z celem minimalizacji strat mocy
problem = LpProblem("Minimal Power Consumption Problem", LpMinimize)

# Tworzenie problemu optymalizacyjnego z celem minimalizacji strat mocy
currents = [LpVariable(f"I{i}", lowBound=0) for i in range(5)]
# Tworzenie zmiennych reprezentujących moc na każdej gałęzi obwodu (moc nie może być ujemna)
powers = [LpVariable(f"P{i}", lowBound=0) for i in range(5)]

# Funkcja celu: minimalizacja sumy prądów (co odpowiada minimalizacji strat mocy)
problem += lpSum(currents)
problem += lpSum(powers)

# Ograniczenia: zależność między napięciem, prądem i konduktancją (I = G * U)
problem += voltage[0]*conductance[0]==currents[0]
problem += voltage[1]*conductance[1]==currents[1]
problem += voltage[2]*conductance[2]==currents[2]
problem += voltage[3]*conductance[3]==currents[3]
problem += voltage[4]*conductance[4]==currents[4]

# Definiowanie mocy jako iloczynu kwadratu napięcia i konduktancji (P = U² * G)
problem += voltage_squared[0]*conductance[0]==powers[0]
problem += voltage_squared[1]*conductance[1]==powers[1]
problem += voltage_squared[2]*conductance[2]==powers[2]
problem += voltage_squared[3]*conductance[3]==powers[3]
problem += voltage_squared[4]*conductance[4]==powers[4]

# Ograniczenia związane z zależnością prądów w obwodzie (np. prawo Kirchhoffa)
problem += currents[2]+currents[3]==currents[0]
problem += currents[1]+currents[2]==currents[4]


# Ograniczenia dotyczące maksymalnych wartości prądów w rezystorach
problem += currents[0] <= max_currents[0]
problem += currents[1] <= max_currents[1]
problem += currents[2] <= max_currents[2]
problem += currents[3] <= max_currents[3]
problem += currents[4] <= max_currents[4]

# Ograniczenia dotyczące minimalnych wartości prądów w rezystorach
problem += min_currents[0] <= currents[0]
problem += min_currents[1] <= currents[1]
problem += min_currents[2] <= currents[2]
problem += min_currents[3] <= currents[3]
problem += min_currents[4] <= currents[4]

# Rozwiązanie problemu optymalizacyjnego
problem.solve()

# Wyświetlenie statusu rozwiązania (np. "Optimal" jeśli udało się znaleźć optymalne rozwiązanie)
print("Status:", LpStatus[problem.status])

# Wyświetlenie wartości optymalnych dla prądów i mocy w każdej gałęzi
for i in range(5):
  print(f"Current through resistor {i+1}: {currents[i].varValue} Amps")
  print(f"Power on branch {i+1}: {powers[i].varValue} Watts")


# Wyświetlenie minimalnej całkowitej mocy strat
print(f"Minimum total power: {value(problem.objective)} Watts")

Status: Infeasible
Current through resistor 1: 3.0 Amps
Power on branch 1: 18.0 Watts
Current through resistor 2: 2.0 Amps
Power on branch 2: 10.0 Watts
Current through resistor 3: 1.0 Amps
Power on branch 3: 4.0 Watts
Current through resistor 4: 2.0 Amps
Power on branch 4: 7.0 Watts
Current through resistor 5: 3.0 Amps
Power on branch 5: 9.0 Watts
Minimum total power: 48.0 Watts
